In [2]:
from keras.models import Model
from keras.layers import Dense, Activation, LSTM, GRU, Dropout, Input
from keras.optimizers import SGD, Adam
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
% matplotlib inline

import tensorflow as tf
import json

from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
# Load data
with open('TrainSet_2015_Train1_result.text','r') as f:
    result01= json.load(f)
    
with open('TrainSet_2015_Train2_result.text','r') as f:
    result02 = json.load(f)

with open('TrainSet_2017_Train1_result.text', 'r') as f:
    result03 = json.load(f)

In [4]:
# Reframe data dataset
X1 = np.array([result01[i]['data'] for i in result01.keys()])
X2 = np.array([result02[i]['data'] for i in result02.keys()])
X3 = np.array([result03[i]['data'] for i in result03.keys()])


        
# Drop some data which have too few information in X2
index02 = []
for i in range(len(X2)):
    if len(X2[i]) < 240:
        index02.append(i)
    elif len(X2[i]) != 247 and (247-len(X2[i]))%2 == 0:
        X2[i] = np.pad(X2[i],int((247-len(X2[i]))/2),"edge")
    elif len(X2[i]) != 247 and (247-len(X2[i]))%2 == 1:
        X2[i] = np.pad(X2[i],(int((247-len(X2[i])+1)/2), int((247-len(X2[i])+1)/2)-1),"edge")

new_X2 = np.delete(X2,index02,0)
X2 = np.array([i for i in new_X2])

# Drop some data which have too few information
index03 = []
for i in range(len(X3)):
    if len(X3[i]) < 240:
        index03.append(i)
    elif len(X3[i]) != 247 and (247-len(X3[i]))%2 == 0:
        X3[i] = np.pad(X3[i],int((247-len(X3[i]))/2),"edge")
    elif len(X3[i]) != 247 and (247-len(X3[i]))%2 == 1:
        X3[i] = np.pad(X3[i],(int((247-len(X3[i])+1)/2), int((247-len(X3[i])+1)/2)-1),"edge")

new_X3 = np.delete(X3, index03,0)
X3 = np.array([i for i in new_X3])

# Reframe label dataset
Y1 = np.array([result01[i]['label'] for i in result01.keys()])
Y2 = np.array([result02[i]['label'] for i in result02.keys()])
Y3 = np.array([result03[i]['label'] for i in result03.keys()])

Y2 = np.delete(Y2,index02,0)
Y3 = np.delete(Y3,index03,0)

In [5]:
# concatenate three arrays
X = np.concatenate([X1,X2,X3],axis=0)
Y = np.concatenate([Y1,Y2,Y3],axis=0)

In [6]:
# perform various feature engineering and data preprocessing
def one_hot(Y):
    result = np.zeros((Y.shape[0],3))
    for i in range(Y.shape[0]):
        if int(Y[i]) == -1:
            result[i,0] = 1
        if int(Y[i]) == 0:
            result[i,1] = 1
        if int(Y[i]) == 1:
            result[i,2] = 1
    return result

def max_min_scaling(x):
    minimum = min(x)
    range_x = max(x)-min(x)
    result = []
    for i in x:
        result.append((i-minimum)/range_x)
    result = np.array(result)
    return result

X = np.array([max_min_scaling(i) for i in X])
Y = one_hot(Y)


In [7]:
# Reshape the input data as well as the output data
X = X.reshape(X.shape[0],X.shape[1],1)

In [8]:
# Split the dataset into cross-validation
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [9]:
X_train.shape, y_train.shape

((476, 247, 1), (476, 3))

In [12]:
# Model 

def model(input_shape,n):

    """
        Input shape (247,m)
    """
    X_input = Input(input_shape) 

    X = LSTM(n,return_sequences=True)(X_input)
    X = Activation('relu')(X)
    X = LSTM(n,return_sequences=False)(X)
    X = Activation('relu')(X)

    outputs = Dense(3,activation='sigmoid')(X)

    model = Model(inputs = X_input, outputs = outputs)

    return model

model = model((247,1),25)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=False)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])


In [13]:
# fit the model, use full batch
# run around 1000 epochs

model.fit(X_train,y_train,epochs=1000,batch_size=X_train.shape[0])


Epoch 1/1000
476/476 [==============================] - 3s 7ms/step - loss: 0.6923 - acc: 0.5364
Epoch 2/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6818 - acc: 0.6667
Epoch 3/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6664 - acc: 0.6667
Epoch 4/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6436 - acc: 0.6667
Epoch 5/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6327 - acc: 0.6667
Epoch 6/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6325 - acc: 0.6667
Epoch 7/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6309 - acc: 0.6667
Epoch 8/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6284 - acc: 0.6667
Epoch 9/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6279 - acc: 0.6667
Epoch 10/1000
476/476 [==============================] - 1s 2ms/step - loss: 0.6286 - acc: 0.6667
Epoch 11/1000
476/476 [======

In [28]:
model.evaluate(X_test,y_test)

119/119 [==============================] - 0s 4ms/step


[0.9387000603836124, 0.6918767370095774]

In [27]:
predict = model.predict(X_test) > 0.5